Reproduce plots from: 
The Representation of Sea Salt Aerosols and Their Role in Polar Climate Within CMIP6

GOAL:
- make familiar with Wilcoxon test and dask
- compare parallelized version



(0) Define and regrid the data on a common grid

In [2]:
import os
os.getcwd()

'/home/jovyan/ArditArifi'

In [3]:
source_path="/home/jovyan/prepared-data/POLAR_EMISS_DATA/CMIP6"
processed_path="/home/jovyan/student-storages/GROUP3/ArditArifi/CMIP6_processed"
output_path="/home/jovyan/student-storages/GROUP3/ArditArifi/CMIP6_output"

! mkdir -p {processed_path}
! mkdir -p {output_path}

In [6]:
grid_description = """
gridtype = lonlat
xsize = 120
ysize = 90
xfirst = 0.0
xinc = 3.0
yfirst = -90.0
yinc = 2.0
"""

grid_file = "grid_3x2.txt"

# Save the grid description to a file
with open(grid_file, "w") as grid_file:
    grid_file.write(grid_description)

print("Grid description file created: grid_3x2.txt")

Grid description file created: grid_3x2.txt


In [9]:
var = "tas"
! ./reproduce.sh {source_path} {output_path} {var}

expand_wild_cards: Argument '{source_path}/tas_*.nc' contains one of the following unsupported unquoted characters: <newline>, `|', `&', `;', `<', `>', `(', `)', `{', `}'.
Regridded: {source_path}/tas_*.nc -> {output_path}/tas_*.nc


(1) Reproduce Fig 10.

In [21]:
import glob
import xarray as xr


model = "NorESM2"


# Define the pattern

piClim_2xss = xr.open_mfdataset( f"{source_path}/rtmt*{model}*piClim-2xss*.nc", combine="by_coords")
piClim_control = xr.open_mfdataset( f"{source_path}/rtmt*{model}*piClim-control*.nc", combine="by_coords")

SyntaxError: unmatched '}' (373890938.py, line 12)

<xarray.Dataset> Size: 21MB
Dimensions:    (time: 360, bnds: 2, lat: 96, lon: 144)
Coordinates:
  * time       (time) object 3kB 0001-01-16 12:00:00 ... 0030-12-16 12:00:00
  * lat        (lat) float64 768B -90.0 -88.11 -86.21 ... 86.21 88.11 90.0
  * lon        (lon) float64 1kB 0.0 2.5 5.0 7.5 ... 350.0 352.5 355.0 357.5
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) object 6kB dask.array<chunksize=(1, 2), meta=np.ndarray>
    lat_bnds   (time, lat, bnds) float64 553kB dask.array<chunksize=(120, 96, 2), meta=np.ndarray>
    lon_bnds   (time, lon, bnds) float64 829kB dask.array<chunksize=(120, 144, 2), meta=np.ndarray>
    rtmt       (time, lat, lon) float32 20MB dask.array<chunksize=(1, 96, 144), meta=np.ndarray>
Attributes: (12/49)
    Conventions:               CF-1.7 CMIP-6.2
    activity_id:               AerChemMIP
    branch_method:             Hybrid-restart from year 1751-01-01 of piControl
    branch_time:               0.0
    branch_time_in_child:      0.0
    branch_time_in_parent:     485450.0
    ...                        ...
    title:                     NorESM2-LM output prepared for CMIP6
    variable_id:               rtmt
    variant_label:             r1i1p1f1
    license:                   CMIP6 model data produced by NCC is licensed u...
    cmor_version:              3.5.0
    tracking_id:               hdl:21.14100/8c472e7a-0fc6-4b88-9548-f5c775069af0

In [25]:
piClim_2xss.time.values

array([cftime.DatetimeNoLeap(1, 1, 16, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1, 2, 15, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1, 3, 16, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1, 4, 16, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1, 5, 16, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1, 6, 16, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1, 7, 16, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1, 8, 16, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1, 9, 16, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1, 10, 16, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1, 11, 16, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1, 12, 16, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2, 1, 16, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2, 2, 15, 0, 0, 0, 0, ha